<a href="https://colab.research.google.com/github/bflatau/codeart_io/blob/add_embeddings/DEEP_THOUGHT_Obtain_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### DATA

Use this file: https://drive.google.com/file/d/1Hiwc-OfExpxQ1_gxbBELk1LyY7rXHjQk/view?usp=sharing

### SETUP OPENAI

In [ ]:
# INSTALL OPENAI LIBRARY
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 40.2 MB/s 
  Created wheel for openai: filename=openai-0.19.0-py3-none-any.whl size=53535 sha256=b15e3eb517e613b980df7ecf85b0224d80756510f06076191581b95522e920c0
  Stored in directory: /root/.cache/pip/wheels/94/b5/c0/928013bd6418b23b9c5d89fb24cdeb1faae899c11377d69609
Successfully built openai


In [ ]:
import openai

# Load your API key from an environment variable or secret management service
#openai.api_key = ##APIKEYHERE##

## 1. Load the dataset

The dataset used in this example is [fine-food reviews](https://www.kaggle.com/snap/amazon-fine-food-reviews) from Amazon. The dataset contains a total of 568,454 food reviews Amazon users left up to October 2012. We will use a subset of this dataset, consisting of 1,000 most recent reviews for illustration purposes. The reviews are in English and tend to be positive or negative. Each review has a ProductId, UserId, Score, review title (Summary) and review body (Text).

We will combine the review summary and review text into a single combined text. The model will encode this combined text and it will output a single vector embedding.

In [9]:
import pandas as pd

df = pd.read_csv('combined_season1-37.tsv',  sep='\t') #read file
df = df[[ 'answer', 'question']] #name the columns we want
df = df.dropna() #remove all the columns not identified above
df['combined'] = "Question: " + df.question.str.strip() + "; Answer: " + df.answer.str.strip() #make superstring?
#df.head(6) #this tests N rows to see if the data looks correct

df = df.tail(1000) # set df to the last N number of values to work with
#len(df) #output length



###REFERENCE
#df = pd.read_csv('input/Reviews.csv', index_col=0)
#df = df[['Time', 'ProductId', 'UserId', 'Score', 'Summary', 'Text']]
#df = df.dropna()
#df['combined'] = "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
#df.head(2)

## subsample to 1k most recent reviews and remove samples that are too long
#df = df.sort_values('Time').tail(1_100)
#df.drop('Time', axis=1, inplace=True)

#from transformers import GPT2TokenizerFast
#tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

## remove reviews that are too long
#df['n_tokens'] = df.combined.apply(lambda x: len(tokenizer.encode(x)))
#df = df[df.n_tokens<2000].tail(1_000)
#len(df)

,answer,question,combined
389435,"Idiomatically, if you've ""lost"" these small sp...",marbles,"Question: marbles; Answer: Idiomatically, if y..."
389436,"Something overlooked has been ""lost in"" this, ...",the shuffle,Question: the shuffle; Answer: Something overl...
389437,Art Fleming & Don Pardo were in the video for ...,"\""I Lost On Jeopardy\""","Question: \""I Lost On Jeopardy\""; Answer: Art ..."
389438,"In ""A Moveable Feast"", Gertrude Stein is quote...",Lost Generation,"Question: Lost Generation; Answer: In ""A Movea..."
389439,The biblical Asher & Zebulun were 2 of these t...,the lost tribes of Israel,Question: the lost tribes of Israel; Answer: T...
389440,This hefty noisemaker from Whitechapel Foundry...,Big Ben,Question: Big Ben; Answer: This hefty noisemak...
389441,"Around 4,000 years ago, the first foundries in...",bronze,"Question: bronze; Answer: Around 4,000 years a..."
389442,Several different foundries worked for 4 month...,Monitor,Question: Monitor; Answer: Several different f...
389443,"Once one of the largest of its kind, the Gary ...",U.S. Steel,Question: U.S. Steel; Answer: Once one of the ...
389444,2 of the 3 women depicted on the first statue ...,"(2 of) (Sojourner) Truth, (Susan B.) Anthony, ...","Question: (2 of) (Sojourner) Truth, (Susan B.)..."


### 2. Get embeddings and save them for future reuse

In [ ]:
from openai.embeddings_utils import get_embedding

# This will take just under 10 minutes
df['babbage_similarity'] = df.combined.apply(lambda x: get_embedding(x, engine='text-similarity-babbage-001'))
df['babbage_search'] = df.combined.apply(lambda x: get_embedding(x, engine='text-search-babbage-doc-001'))
df.to_csv('embedded_1k_reviews.csv')